<a href="https://colab.research.google.com/github/jboca-andes/Recursos/blob/main/Tarea_calificada_Perfilamiento_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar xf spark-3.2.0-bin-hadoop2.7.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

In [3]:
!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 55.4 MB/s 


In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
aeropuertos = 'aeropuertos.csv'
vuelos = 'vuelos.csv'
from pyspark import SparkFiles
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%201/{aeropuertos}')
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%201/{vuelos}')

In [21]:
df_aeropuertos = spark.read.csv('file://'+SparkFiles.get(aeropuertos), header=True, inferSchema=True)
df_vuelos = spark.read.csv('file://'+SparkFiles.get(vuelos), header=True, inferSchema=True)

df_aeropuertos.createOrReplaceTempView("aeropuertos")
df_vuelos.createOrReplaceTempView("vuelos")

## Pregunta:
Visualizar el número de pasajeros transportados en un rango de fechas de aeropuertos Internacionales a Colombia

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de los pasajeros transportados desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [35]:
# Pasajeros transportados desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(pasajeros) pasajeros_transportados from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|pasajeros_transportados|
+----+---+-----------------------+
|2012| 10|              2107605.0|
|2010| 12|              2023021.0|
|2010|  7|              1842447.0|
|2013|  2|              1648071.0|
|2012|  8|              1906273.0|
|2012|  4|              1702058.0|
|2012| 12|              2358191.0|
|2013|  9|              1904437.0|
|2013| 10|              2303307.0|
|2013| 12|              1981243.0|
+----+---+-----------------------+
only showing top 10 rows



In [36]:
# Pasajeros transportados hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(pasajeros) pasajeros_transportados from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|pasajeros_transportados|
+----+---+-----------------------+
|2012| 10|              2027717.0|
|2010|  7|              1854910.0|
|2010| 12|              1968860.0|
|2012|  8|              1795754.0|
|2013|  2|              1751075.0|
|2012|  4|              1629414.0|
|2012| 12|              2224497.0|
|2013|  9|              1981828.0|
|2013| 10|              2268371.0|
|2013| 12|              2004059.0|
+----+---+-----------------------+
only showing top 10 rows



## Pregunta:
***Visualizar la carga transportada en un rango de fechas de aeropuertos Internacionales a Colombia***

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de la carga transportada desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [43]:
# Carga transportada desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_bordo) carga from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.carga_bordo >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|               carga|
+----+---+--------------------+
|2012| 10|1.0652734819999995E7|
|2010|  7|          8999624.47|
|2010| 12|       1.630879511E7|
|2013|  2|   7831548.309999999|
|2012|  8|   8871730.169999998|
|2012|  4|1.0231764119999997E7|
|2012| 12|1.3609474729999997E7|
|2013|  9|1.5701067399999999E7|
|2013| 10|1.5676513499999998E7|
|2013| 12|       1.647605157E7|
+----+---+--------------------+
only showing top 10 rows



In [44]:
# Carga transportada hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_bordo) carga from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.carga_bordo >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|               carga|
+----+---+--------------------+
|2012| 10|   9094384.079999998|
|2010|  7|          7420303.08|
|2010| 12|1.3420641440000001E7|
|2012|  8|   7153734.279999999|
|2013|  2|   7196776.220000001|
|2012|  4|   9038773.819999998|
|2012| 12|1.1701853099999998E7|
|2013|  9|1.4805132079999998E7|
|2013| 10|1.2822389030000001E7|
|2013| 12|       1.500696409E7|
+----+---+--------------------+
only showing top 10 rows



## Pregunta:
***Visualizar la carga desaprovechada en un rango de fechas de aeropuertos Internacionales a Colombia***

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de la carga desaprovechada desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [45]:
# Carga desaprovechada hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_ofrecida - carga_bordo) carga_desaprovechada from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.carga_ofrecida >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|carga_desaprovechada|
+----+---+--------------------+
|2012| 10|1.2467720043999998E8|
|2010|  7|       5.361381257E7|
|2010| 12|      1.3593209244E8|
|2012|  8| 9.274949392999999E7|
|2013|  2| 8.566440394000001E7|
|2012|  4| 8.837291497999999E7|
|2012| 12|      1.1447950613E8|
|2013|  9|       8.703813912E7|
|2013| 10|1.1072354300999999E8|
|2013| 12|1.1420259094000003E8|
+----+---+--------------------+
only showing top 10 rows



In [46]:
# Carga desaprovechada desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_ofrecida - carga_bordo) carga_desaprovechada from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.carga_ofrecida >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|carga_desaprovechada|
+----+---+--------------------+
|2012| 10|      1.2357486432E8|
|2010|  7|       5.227463391E7|
|2010| 12|1.1299611522999999E8|
|2013|  2|       7.952276012E7|
|2012|  8|       9.104578136E7|
|2012|  4| 9.270638188999996E7|
|2012| 12|       1.111906144E8|
|2013|  9|       9.182402302E7|
|2013| 10|1.0880657265999998E8|
|2013| 12|1.0423506235000001E8|
+----+---+--------------------+
only showing top 10 rows



## Pregunta:
***Visualizar el número de puestos desaprovechados en un rango de fechas de aeropuertos Internacionales a Colombia***

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de los puestos desaprovechados desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [48]:
# Puestos desaprovechados desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(sillas - pasajeros) puestos_desaprovechados from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.sillas >= 0 and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|puestos_desaprovechados|
+----+---+-----------------------+
|2012| 10|               458819.0|
|2010| 12|               419584.0|
|2010|  7|               328342.0|
|2013|  2|               357415.0|
|2012|  8|               365439.0|
|2012|  4|               564234.0|
|2012| 12|               425236.0|
|2013|  9|               366431.0|
|2013| 10|               450551.0|
|2013| 12|               351656.0|
+----+---+-----------------------+
only showing top 10 rows



In [49]:
# Puestos desaprovechados hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(sillas - pasajeros) puestos_desaprovechados from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.sillas >= 0 and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|puestos_desaprovechados|
+----+---+-----------------------+
|2012| 10|               441357.0|
|2010|  7|               301175.0|
|2010| 12|               562669.0|
|2012|  8|               335900.0|
|2013|  2|               355057.0|
|2012|  4|               537719.0|
|2012| 12|               514333.0|
|2013|  9|               360373.0|
|2013| 10|               432513.0|
|2013| 12|               452507.0|
+----+---+-----------------------+
only showing top 10 rows



In [50]:
spark.sql("select distinct categoria from aeropuertos").show()

+-------------+
|    categoria|
+-------------+
|     Nacional|
|Internacional|
|    Aeródromo|
|     Regional|
+-------------+



In [23]:
spark.sql("select * from aeropuertos where categoria = 'Internacional' limit 10").show()

+---+-----+----+--------------------+-------------------+------------------+-------------+-------+--------+-----------+--------------------+--------------+-----------+----+---------+----------+------------------+--------------+-----+---------+--------------------+----------------+-------------+
|_c0|sigla|iata|              nombre|          municipio|      departamento|    categoria|latitud|longitud|propietario|          explotador|longitud_pista|ancho_pista|pbmo|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|     tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+---+-----+----+--------------------+-------------------+------------------+-------------+-------+--------+-----------+--------------------+--------------+-----------+----+---------+----------+------------------+--------------+-----+---------+--------------------+----------------+-------------+
|274|  BOG| BOG|           EL DORADO|       Bogotá, D.C.|      Bogotá, D.C.|Internacional| 4.7017|-74.1469|  AER

In [27]:
spark.sql("select * from vuelos limit 10").show()

+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+-----------+
| ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|             empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+-----------+
|2012|  1|   BOG|    CUC|       BE20|         T|      N|             RIO SUR|   1.0|   0.0|           0.0|      4.0|      100.0|
|2013|  5|   UIB|    BOG|       DH8D|         R|      N|               AIRES|  30.0|1110.0|       24000.0|    873.0|     4222.0|
|2013| 10|   IBE|    BOG|       DH8D|         R|      N|               AIRES|  98.0|3626.0|       56056.0|   2866.0|    2323.75|
|2012|  4|   FLA|    BOG|       JS32|         T|      N|         SARPA S.A.S|   1.0|   0.0|           0.0|      4.0|        0.0|
|2013|  7|   CUC|    AUC|       C402|         T|      N|AEROCHARTER ANDIN...|   1.0|   0.0|      